In [1]:

import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import pandas as pd
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [0]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    print
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [0]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s),padding='valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    
    return X

In [0]:
# GRADED FUNCTION: ResNet50

def ResNet18(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')


    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')


    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet18')

    return model

In [5]:
model = ResNet18(input_shape = (48, 48, 3), classes = 7)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [0]:
!tar -xf "/content/fer2013.tar.gz"

In [10]:
from keras.utils import np_utils
data = pd.read_csv('/content/fer2013/fer2013.csv')
train = data[data.Usage == "Training"]
test = data[data.Usage == "PrivateTest"]
test_pub = data[data.Usage == "PublicTest"]


train_labels = train["emotion"].values
test_labels = test["emotion"].values
test_pub_labels = test_pub["emotion"].values

train_ds = train["pixels"].values
test_ds = test["pixels"].values
test_pub_ds = test_pub["pixels"].values

train_ds_list = list()
test_ds_list = list()
test_pub_ds_list = list()

for i in range(28709):
  train_ds_list.append(np.array(train_ds[i].split(" ")))
  train_ds_list[i] = train_ds_list[i].astype(np.float)

for i in range(3589):
  test_ds_list.append(np.array(test_ds[i].split(" ")))
  test_ds_list[i] = test_ds_list[i].astype(np.float)

for i in range(3589):
  test_pub_ds_list.append(np.array(test_pub_ds[i].split(" ")))
  test_pub_ds_list[i] = test_pub_ds_list[i].astype(np.float)

train_np = np.array(train_ds_list)
test_np = np.array(test_ds_list)
test_pub_np = np.array(test_pub_ds_list)



train_np = train_np.reshape(train_np.shape[0], 48, 48, 1)
test_np = test_np.reshape(test_np.shape[0],48, 48, 1)
test_pub_np = test_pub_np.reshape(test_pub_np.shape[0], 48, 48, 1)

train_np = np.repeat(train_np , 3, axis=3)
test_np = np.repeat(test_np , 3, axis=3)
test_pub_np = np.repeat(test_pub_np , 3, axis=3)
print(train_np.shape)

train_labels = np_utils.to_categorical(train_labels, 7)
test_labels = np_utils.to_categorical(test_labels, 7)
test_pub_labels = np_utils.to_categorical(test_pub_labels, 7) 


(28709, 48, 48, 3)


In [0]:
from keras.preprocessing.image import ImageDataGenerator
# parameters
num_epochs = 100
verbose = 1
batch_size = 64


train_big = np.vstack([train_np, test_pub_np])
train_label_big = np.concatenate([train_labels, test_pub_labels])
# data generator
data_generator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [0]:
model.save('Resnet18_model.hdf5')

In [16]:
model.fit_generator(data_generator.flow(train_big, train_label_big),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=num_epochs, verbose=1, 
                    validation_data=data_generator.flow(test_np, test_labels)
)

Epoch 1/100
505/504 [==============================] - 27s 54ms/step - loss: 1.8843 - acc: 0.2394 - val_loss: 1.8680 - val_acc: 0.2379
Epoch 2/100
505/504 [==============================] - 27s 53ms/step - loss: 1.8989 - acc: 0.2406 - val_loss: 1.9603 - val_acc: 0.2438
Epoch 3/100
505/504 [==============================] - 27s 53ms/step - loss: 1.8939 - acc: 0.2433 - val_loss: 1.8798 - val_acc: 0.2416
Epoch 4/100
505/504 [==============================] - 27s 53ms/step - loss: 1.8555 - acc: 0.2405 - val_loss: 1.8373 - val_acc: 0.2491
Epoch 5/100
505/504 [==============================] - 27s 53ms/step - loss: 1.8399 - acc: 0.2452 - val_loss: 1.8183 - val_acc: 0.2488
Epoch 6/100
505/504 [==============================] - 27s 52ms/step - loss: 1.8044 - acc: 0.2533 - val_loss: 1.7854 - val_acc: 0.2664
Epoch 7/100
505/504 [==============================] - 26s 52ms/step - loss: 1.7901 - acc: 0.2598 - val_loss: 1.8006 - val_acc: 0.2488
Epoch 8/100
505/504 [==============================] - 

In [0]:
model.save('Resnet18_model.hdf5')
model.save_weights("weights_100_epochs_Resnet18.hdf5")


In [18]:
model.summary()

Model: "ResNet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 24, 24, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 24, 24, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
data2 = pd.read_csv('/content/fer2013/fer2013.csv')
data2 = data2.loc[ data2['emotion'].isin([0,3,4,6])]
data2.loc[(data2['emotion'] == 3),'emotion'] = 1
data2.loc[(data2['emotion'] == 4),'emotion'] = 2
data2.loc[(data2['emotion'] == 6),'emotion'] = 3

In [22]:
train = data2.loc[ data2['Usage'] == "Training"]
test = data2[data2.Usage == "PrivateTest"]
test_pub = data2[data2.Usage == "PublicTest"]

train_labels = train["emotion"].values
test_labels = test["emotion"].values
test_pub_labels = test_pub["emotion"].values

train_ds = train["pixels"].values
test_ds = test["pixels"].values
test_pub_ds = test_pub["pixels"].values

train_ds_list = list()
test_ds_list = list()
test_pub_ds_list = list()
print(train_ds.size)
print(test_ds.size)
print(test_pub_ds.size)
for i in range(train_ds.size):
  train_ds_list.append(np.array(train_ds[i].split(" ")))
  train_ds_list[i] = train_ds_list[i].astype(np.float)

for i in range(test_ds.size):
  test_ds_list.append(np.array(test_ds[i].split(" ")))
  test_ds_list[i] = test_ds_list[i].astype(np.float)

for i in range(test_pub_ds.size):
  test_pub_ds_list.append(np.array(test_pub_ds[i].split(" ")))
  test_pub_ds_list[i] = test_pub_ds_list[i].astype(np.float)

train_np = np.array(train_ds_list)
test_np = np.array(test_ds_list)
test_pub_np = np.array(test_pub_ds_list)



train_np = train_np.reshape(train_np.shape[0], 48, 48, 1)
test_np = test_np.reshape(test_np.shape[0],48, 48, 1)
test_pub_np = test_pub_np.reshape(test_pub_np.shape[0], 48, 48, 1)


train_np = np.repeat(train_np , 3, axis=3)
test_np = np.repeat(test_np , 3, axis=3)
test_pub_np = np.repeat(test_pub_np , 3, axis=3)


train_labels = np_utils.to_categorical(train_labels, 4)
test_labels = np_utils.to_categorical(test_labels, 4)
test_pub_labels = np_utils.to_categorical(test_pub_labels, 4) 

patience = 50
batch_size = 64
num_epochs = 60


train_big = np.vstack([train_np, test_pub_np])
train_label_big = np.concatenate([train_labels, test_pub_labels])

21005
2590
2622


In [0]:
data_generator = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [31]:
model = ResNet18(input_shape = (48, 48, 3), classes = 4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(data_generator.flow(train_big, train_label_big),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=100, verbose=1, 
                    validation_data=data_generator.flow(test_np, test_labels)
)

Epoch 1/100
370/369 [==============================] - 30s 82ms/step - loss: 2.0457 - acc: 0.2922 - val_loss: 1.7743 - val_acc: 0.3371
Epoch 2/100
370/369 [==============================] - 20s 55ms/step - loss: 1.7173 - acc: 0.3185 - val_loss: 1.8077 - val_acc: 0.3066
Epoch 3/100
370/369 [==============================] - 21s 56ms/step - loss: 1.6454 - acc: 0.3331 - val_loss: 1.4458 - val_acc: 0.3375
Epoch 4/100
370/369 [==============================] - 21s 57ms/step - loss: 1.4516 - acc: 0.3422 - val_loss: 1.7034 - val_acc: 0.3301
Epoch 5/100
370/369 [==============================] - 21s 57ms/step - loss: 1.4860 - acc: 0.3411 - val_loss: 1.4251 - val_acc: 0.3347
Epoch 6/100
370/369 [==============================] - 21s 56ms/step - loss: 1.4205 - acc: 0.3454 - val_loss: 1.4837 - val_acc: 0.3425
Epoch 7/100
370/369 [==============================] - 21s 56ms/step - loss: 1.4113 - acc: 0.3415 - val_loss: 1.3876 - val_acc: 0.3425
Epoch 8/100
370/369 [==============================] - 